WARNING: experimental FSI + .net standard

In [1]:
#load "Paket.fsx"

In [2]:
Paket.Version [ 
    "Microsoft.ML", "0.5"
    "NETStandard.Library", "2.0.3"]

In [3]:
//Patch location for native dependencies
System.Environment.SetEnvironmentVariable("Path", 
    System.Environment.GetEnvironmentVariable("Path") + ";" + Paket.RootPath + @"/packages/Microsoft.ML/runtimes/win-x64/native/")
    
System.Environment.SetEnvironmentVariable("Path", 
    System.Environment.GetEnvironmentVariable("Path") + ";" + Paket.RootPath + @"/packages/Microsoft.ML.CpuMath/runtimes/win-x64/native/")

In [4]:
#r "packages/NETStandard.Library/build/netstandard2.0/ref/netstandard"
#load "Paket.Generated.Refs.fsx"

In [5]:
open Microsoft.ML
open Microsoft.ML.Runtime.Api
open Microsoft.ML.Transforms
open Microsoft.ML.Trainers
open System
open Microsoft.ML.Data

In [6]:
Util.Url "https://github.com/dotnet/machinelearning/blob/master/test/data/wikipedia-detox-250-line-data.tsv"

{ContentType = "text/html; charset=utf-8";
 Data =
  [|10uy; 10uy; 10uy; 10uy; 10uy; 10uy; 60uy; 33uy; 68uy; 79uy; 67uy; 84uy; 89uy;
    80uy; 69uy; 32uy; 104uy; 116uy; 109uy; 108uy; 62uy; 10uy; 60uy; 104uy; 116uy;
    109uy; 108uy; 32uy; 108uy; 97uy; 110uy; 103uy; 61uy; 34uy; 101uy; 110uy;
    34uy; 62uy; 10uy; 32uy; 32uy; 60uy; 104uy; 101uy; 97uy; 100uy; 62uy; 10uy;
    32uy; 32uy; 32uy; 32uy; 60uy; 109uy; 101uy; 116uy; 97uy; 32uy; 99uy; 104uy;
    97uy; 114uy; 115uy; 101uy; 116uy; 61uy; 34uy; 117uy; 116uy; 102uy; 45uy;
    56uy; 34uy; 62uy; 10uy; 32uy; 32uy; 60uy; 108uy; 105uy; 110uy; 107uy; 32uy;
    114uy; 101uy; 108uy; 61uy; 34uy; 100uy; 110uy; 115uy; 45uy; 112uy; 114uy;
    101uy; 102uy; 101uy; 116uy; 99uy; 104uy; ...|];}

In [7]:
let testDataPath = @"wikipedia-detox-250-line-data.tsv"

In [8]:
type SentimentData() =
    [<Column(ordinal = "0", name = "Label"); DefaultValue>]
    val mutable Sentiment : float32
    [<Column(ordinal = "1"); DefaultValue>]
    val mutable SentimentText : string

type SentimentPrediction() =
    [<ColumnName "PredictedLabel"; DefaultValue>]
    val mutable Sentiment : bool

In [9]:
let pipeline = LearningPipeline()
pipeline.Add(TextLoader(testDataPath).CreateFrom<SentimentData>())
pipeline.Add(TextFeaturizer("Features", "SentimentText"))
pipeline.Add(FastTreeBinaryClassifier(NumLeaves = 5, NumTrees = 5, MinDocumentsInLeafs = 2))

In [10]:
let model = pipeline.Train<SentimentData, SentimentPrediction>()

Processed 251 rows with 0 bad values and 1 format errors
Processed 251 rows with 0 bad values and 1 format errors
Processed 251 rows with 0 bad values and 1 format errors
Not adding a normalizer.
Making per-feature arrays
Changing data from row-wise to column-wise
  Bad value at line 1 in column Label
Processed 251 rows with 1 bad values and 1 format errors
Processed 250 instances
Binning and forming Feature objects
Reserved memory for tree learner: 2020236 bytes
Starting to train ...
Not training a calibrator because it is not needed.


In [23]:
//let evaluator = new BinaryClassificationEvaluator();

In [21]:
let predictions =
    [ SentimentData(SentimentText = "Contoso's 11 is a wonderful experience")
      SentimentData(SentimentText = "Oooooh thank you Mr. DietLimeCola. Once again, nice job trying to pretend you have some authority over anybody here. You are a wannabe admin, which is even sadder than a real admin") ]
    |> List.map model.Predict

predictions
|> Seq.iter(fun p -> printfn "%b" p.Sentiment)

true
false


In [19]:
let predictions =
    [ SentimentData(SentimentText = "SOMETHING AWFUL IS DEAD DEAD DEAD DEAD DEAD")]
    |> List.map model.Predict

predictions
|> Seq.iter(fun p -> printfn "%b" p.Sentiment)

true
